<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Getting Started With Linear Regression

_Authors: Kevin Markham (Washington, D.C.), Ed Podojil (New York City)_

**Warning:** Many students find this lesson substantially more challenging than previous lessons in the course. Don't panic if certain parts of it don't click right away. Do put in the time to review, practice, and read about these topics as needed.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd

plt.style.use('fivethirtyeight')

In [ ]:
%matplotlib inline

## Demo

Let's use linear regression to predict prices in the Ames housing dataset.

In [ ]:
# Use pandas to load in the data
ames_df = pd.read_csv('../assets/data/ames_train.csv')

In [ ]:
# To get us started, use just the numeric columns without missing data
ames_df = ames_df.select_dtypes(['int64', 'float64']).dropna(axis='columns').drop('Id', axis='columns')

In [ ]:
# Split the data into the column `y` we want to predict and the 
# columns `X` we will use to make the predictions
X = ames_df.drop('SalePrice', axis='columns')
y = ames_df.loc[:, 'SalePrice']

In [ ]:
# Set aside 25% of the data for testing the model
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
# Import a model class
from sklearn.linear_model import LinearRegression

# Create a model from that class
lr = LinearRegression()

# Ask the model to learn a function that predicts `y` from `X`
lr.fit(X_train, y_train)

That's it! We just built a machine learning model.

Is it any good? Let's test it on data that it hasn't seen.

In [ ]:
# Score the model on the test data
lr.score(X_test, y_test)

This score tells us (roughly) by what factor (0 to 1) our model's errors are on average than the errors that we would get if we just guessed the average price every time. This number is substantially greater than 0, so our model works!

We can also look at the model's predictions and compare them to the actual values:

In [ ]:
list(zip(lr.predict(X_test), y_test))

In [ ]:
# Get the names of the features
X.columns

Linear regression simply takes each of our features, multiplies it by a number, and adds up the result plus an "intercept" term. So in this case,

$$\text{Predicted Price} = \beta_0 + \beta_1 * MSSubClass + \beta_2 * LotArea + \ldots + \beta_{33} YrSold $$

where $\beta_0, \beta_1, \ldots, \beta_{33}$ are numbers that the linear regression algorithm picks for us.

(If you absorbed our lesson on linear algebra, you may see that for a linear regression model, predicted price for a given house is the inner product of our features with a vector of coefficients $[\beta_0, \beta_1, \ldots, \beta_{33}]$, except that we tack on a vector of ones to our features for the intercept term.)

We can actually look at the intercept and coefficients that our algorithm came up with:

In [ ]:
# Print the intercept and the coefficients paired with the corresponding column names
print(lr.intercept_)
print(list(zip(ames_df.columns, lr.coef_)))

You will learn in this lesson how to interpret those numbers.

## The Bikeshare Data Set

We are going to use bikeshare data to build a simple demand forecasting model.

In [ ]:
# Read the data and set the datetime as the index.


In [ ]:
# Preview the first five rows of the DataFrame.


Each observation is an hour of events.

`count` indicates the total number of riders. It will be the target variable that we try to forecast.

`casual` and `registered` are segments of the total users, so they are alternative target variables  that aren't available for forecasting total users.

### Data dictionary

| Variable| Description |
|---------|----------------|
|datetime| hourly date + timestamp  |
|season|  1 = winter, 2 = spring, 3 = summer, 4 = fall |
|holiday| whether the day is considered a holiday|
|workingday| whether the day is neither a weekend nor holiday|
|weather| See below|
|temp| temp_celsius in Celsius|
|atemp| "feels like" temp_celsius in Celsius|
|humidity| relative humidity|
|windspeed| wind speed|
|casual| number of non-num_registered_users user rentals initiated|
|registered| number of num_registered_users user rentals initiated|
|count| number of total rentals|

_Details on Weather Variable_:

- **1**: Clear, Few clouds, Partly cloudy, Partly cloudy
- **2**: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
- **3**: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
- **4**: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog

In [ ]:
# Make names more explicit
bikes = bikes.rename(
    columns={'temp':'temp_celsius',
             'atemp':'atemp_celsius',
             'windspeed': 'windspeed_knots',
             'casual': 'num_casual_users',
             'registered': 'num_registered_users',
             'season': 'season_num',
             'holiday': 'is_holiday',
             'workingday': 'is_workingday',
             'humidity': 'humidity_percent',
             'count': 'num_total_users'
            },
)

## A Linear Model for the Bikeshare Dataset

In [ ]:
# make a scatterplot of `num_total_users` against `temp_celsius`


In [ ]:
# Use Seaborn to create a scatterplot with regression line
# sns.lmplot does not allow us to pass in an Axes object.
# It creates a FaceGrid object that has the relevant Axes object
# as its ax attribute.


We just created a linear regression model!

- **Formula for a line:** $y = mx + b$
- **Alternative notation:** $y = \beta_0 + \beta_1 * x$
- **Our model:** $\mbox{num_total_users} = \beta_0 + \beta_1 * \mbox{temp_celsius} + \epsilon$

We call $\beta_0$ the **model intercept** and $\beta_1$ the **coefficient of `temp_celsius`**.

We call $\epsilon$ the **noise** or **error term**. It accounts for the fact that our points do not lie exactly on a line. Linear regression is designed to be optimal when this noise is normally distributed with constant variance. We ignore it when we use the model to make predictions.

**Exercise (4 mins, in groups).** Answer each question below in terms of $\beta_0$ ("beta naught"), $\beta_1$, and $\epsilon$ ("epsilon").

- What would our model predict for `num_total_users` at `temp_celsius=0`?

- If `temp_celsius` increases by 1, how does our model's prediction for `num_total_users` change? What if $\beta_1$ were negative?

$\blacksquare$

It's important to understand that this model does **not** tell us what would happen if we were to *intervene* to change the temperature in Celsius (say, by deploying thousands of heat lamps around the city). It may be that temperature is correlated with ridership purely because students are more likely to ride when they are on break and they have their longest break during the summer, and not directly because of the temperature. In that case, intervene to change the temperature directly will have no effect.

**Machine learning models trained on passively collected data (as opposed to data from an experiment) cannot be expected to generalize to scenarios in which you are interfering with the system.** We will talk more about this issue in a lesson on causation.

**Takeaways:**

- Linear regression with one input feature chooses the *line* that "best fits" a scatterplot of the target variable against that input feature.
- The model intercept tells you what the model would predict if all of the input variables were zero.
- The coefficient on a variable tells you how much and in what direction the model's prediction would change if the relevant variable were to increase by 1.
- If the data was collected passively (e.g. not as part of a randomized experiment), then a linear regression model is **not** reliable for telling you what would happen if you were to intervene to change the value of a feature.

## Building a Linear Regression Model with scikit-learn

When we called `sns.lmplot`, `seaborn` created a linear regression model which it then displayed to us.

A more typical workflow for creating a linear regression model uses scikit-learn.

scikit-learn is the most popular Python library for machine learning.

**Strengths:**

- Includes good implementations of a wide range of algorithms.
- Provides consistent interface across model types.
- Provides excellent documentation.
- Large community --> tons of resources for learning and getting questions answered.

**Limitations:**

- Designed primarily for single-thread, in-memory computing.
- Has only basic capabilites for deep learning.
- Reflects machine learning rather than statistics mindset: focuses on predictive accuracy on held-out data rather than hypothesis testing, parameter estimation, and model interpretation.

### Preparing the Data

In [ ]:
# 1. Split your feature columns from your target column


In [ ]:
# 2. Refine your feature columns.
# In machine learning competitions, most of the work happens in this step.
# We will talk more about "feature engineering" in another lesson.
# For now, let's just select the temp_celsius column.


In [ ]:
# 3. Split your rows into a training set and a test set.
# Setting a value for `random_state` ensures that we get the same
# result if we run this code more than once, which can be helpful
# for debugging. What we set it to is arbitrary.


### Training the Model

In [ ]:
# 1. Import the LinearRegression model class.


In [ ]:
# 2. Make an instance of the LinearRegression class.


In [ ]:
# 3. Train the model instance on the training set.
# Fitting changes the model `lr_celsius` in-place. scikit-learn is figuring out
# the values to use for $\beta_0$ and $\beta_1$ and storing them inside the model
# so that it can be used for prediction.


### Evaluating the Model

In [ ]:
# Score the model on the test set.


In [ ]:
# Look at the model's errors against time


**Exercise (4 mins.)**

- What would the plot above look like if our model's predictions were perfectly accurate?

- Recall that linear regression is designed to be optimal when errors around a linear model are normally distributed around zero with constant variance. Is that what we see in this case?

$\blacksquare$

### Takeaway: Basic Scikit-Learn Supervised Learning Workflow

- **Prepare the data:**
    - Load it in. Reshape it is necessary so that each row is an observation and each column is a variable.
    - Split columnwise into a target variable and one or more feature variables.
    - Transform your variables to make them more useful to your model.
    - Split rowwise into a training set and a test set.
- **Train the model:**
    - Import the model class.
    - "Instantiate" the class. You can specify "hyperparameters" at this point.
    - Fit the model instance on the training set. (This step changes the model object in-place.)
- **Measure the model's performance on the test set.**

## Linear Regression with Multiple Features

- **Using `temp_celsius` only:** $\mbox{num_total_users} = \beta_0 +  \beta_1 * \mbox{temp_celsius} + \epsilon$
- **Using `temp_celsius` and `humidity_percent`:** $\mbox{num_total_users} = \beta_0 +  \beta_1 * \mbox{temp_celsius} +  \beta_2 * \mbox{humidity_percent} + \epsilon$

Linear regression with two input features chooses the *plane* that "best fits" a 3D scatterplot of the target variable against those input features.

<img src="../assets/images/3d_scatterplot.png" style="height: 350px">

In general, linear regression chooses the "hyperplane" that "best fits" a scatterplot of the target variable against the input features.

**Exercise (7 mins., in groups)** Answer each question below for the second model written out above, in terms of $\beta_0$, $\beta_1$, $\beta_2$, and $\epsilon$ ("epsilon").

- What would our model predict for `num_total_users` at `temp_celsius=0` and `humidity_percent=0`?

- If `temp_celsius` increases by 5 while `percent_humidity` remains the same, how does our model's prediction for `num_total_users` change?

- If `percent_humidity` decreases by 3 while `temp_celsius` remains the same, how does our model's prediction for `num_total_users` change?

- Why does the previous question say "while `temp_celsius` remains the same?"

- What would it mean if a feature had a coefficient of 0?

$\blacksquare$

**Takeaways:** Interpretation of the parameters of a linear regression model does not change as we add more variables, except that each coefficient tells us how the model's prediction would change if the associated variable were to change *while all other variables remained the same*.

Let's build a linear regression model with two features.

In [ ]:
# 1. Split your feature columns from your target column
# We still have `y` from before, but we'll create it again for practice.
# We have to start over with `X` because we kept only "temp_celsius" before.


In [ ]:
# 2. Refine your feature columns.


In [ ]:
# 3. Split your rows into a training set and a test set.


### Training the Model

In [ ]:
# 1. Import the LinearRegression model class.
# We don't have to do this again, but we will for practice.


In [ ]:
# 2. Make an instance of the LinearRegression class.
# We could retrain our original model, but it seems more natural
# to start over with a new model.


In [ ]:
# 3. Train the model instance on the training set.


In [ ]:
# 4. Score the model on the test set.


**Exercise (8 mins., in pairs)**

Build another linear regression model, this time using all of our features (not including except `num_casual_users`, `num_registered_users`, and `num_total_users`) as inputs instead of just `temp_celsius`.

**Model using all columns as given:** $\mbox{num_total_users} = \beta_0 + \beta_1 * \mbox{season_num} + \beta_2 * \mbox{is_holiday} + \beta_3 * \mbox{is_workingday} + \beta_4 * \mbox{weather} + \beta_5 * \mbox{temp_celsius} + \beta_6 * \mbox{atemp_celsius} + \beta_7 * \mbox{humidity_percent} + \beta_8 * \mbox{windspeed_knots} + \epsilon$

- Split your feature columns from your target column.

- Refine your feature columns -- just drop the columns we aren't using.

- Run the cell below to check your work. (You do not have to understand how this code works -- just know that it will raise an error if your X or y has the wrong type or shape.)

In [ ]:
# Tests
assert isinstance(y, pd.Series)
assert isinstance(X, pd.DataFrame)
assert y.shape == (bikes.shape[0],)
assert X.shape == (bikes.shape[0], 8)

- Split your rows into a training set and a test set.

- Let's not import the `LinearRegression` class again. Just make a new instance of it. Call that instance lr_all to distinguish it from our last model.

- Train the model instance on the training set.

- Score the model on the test set.

$\blacksquare$

### Exploring the intercept and coefficients of the linear model

In [ ]:
# Print intercept and coefficients


In [ ]:
# `zip` example


In [ ]:
# Use `zip` to get variable names next to coefficients


**Exercise (1 min.).** How do the model's predictions change for a holiday vs. a non-holiday, all else being equal?

$\blacksquare$

In [ ]:
# Write the dataset to disk with our modifications so that we can pick up
# where we left off in the next module


## A Little Theory

### What "Best Fits" Means

Fitting a linear regression selects the coefficients and intercept that minimize the **sum of squared errors** of the fitted values on the training set.

![Estimating coefficients](../assets/images/estimating_coefficients.png)

In the diagram above:

- The black dots are the **observed values** of x and y.
- The blue line is our **least squares line**.
- The red lines are the **residuals**, which are the vertical distances between the observed values and the least squares line.

### An Overview of Supervised Learning

![Supervised learning diagram](../assets/images/supervised_learning.png)

## Summary

- Linear regression predicts a target variable by multiplying each feature by a coefficient and adding up the results along with an additional intercept term.
    - Fitting a linear regression model chooses the coefficients and intercept that minimize mean squared error on the training set.
    - The intercept tells you what the model predicts when all features are 0.
    - Each coefficient tells you how the model's predictions change when that variables value increases by one and all other variables stay the same.
- In general, a supervised learning algorithm learns a function that it can use to predict the target variable from the input features.
- Scikit-learn supervised learning workflow:
    - Prepare the data:
        - Load it in. Reshape it is necessary so that each row is an observation and each column is a variable.
        - Split columnwise into a target variable and one or more feature variables.
        - Transform your variables to make them more useful to your model.
        - Split rowwise into a training set and a test set.
    - Train the model:
        - Import the model class.
        - "Instantiate" the class. You can specify "hyperparameters" at this point.
        - Fit the model instance on the training set. (This step changes the model object in-place.)
    - Measure the model's performance on the test set.
- A machine learning model trained on passively collected data is not reliable for predicting what will happen if we intervene to change something in the system.